# Reflexion

В данном ноутбуке [Reflexion](https://arxiv.org/abs/2303.11366) мы сделаем граф, который будет отвечать на вопрос, улучшая его итеративно.
У нас будет 3 главных компонента:

1. Агент, отвечающий на вопрос, с самокритикой своих ответов
2. Поиск информации в интернете для улучшения ответов и решения критики
3. Временная память, которая будет хранить критику прошлого ответа (1)

![reflexion diagram](./img/reflexion.png)

Если хотите пропустить инициализацию всех нод и посмотреть результат -> [секция Построение графа](#Построение-графа) below.
## Инициализация

In [ ]:
# %pip install -U --quiet  gigachain gigagraph
# %pip install -U --quiet duckduckgo-search

In [1]:
import datetime
from typing import List, Sequence, TypedDict

from langchain_community.chat_models import GigaChat
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
)
from langchain_core.output_parsers.gigachat_functions import (
    PydanticOutputFunctionsParser,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langsmith import traceable
from search_tools import create_search_tool

from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation

In [2]:
llm = GigaChat(
    ## Тут ваши данные для входа
    profanity_check=False,
    verify_ssl_certs=False,
    timeout=600,
    model="GigaChat-Pro",
    top_p=0.8,
)

## Агенты
Главными компонентами у нас являются, агенты, которые отвечают на вопрос исходя из своих знаний и информации из интернета, постоянно критикующие свои ответы и улучщающие их итеративно.
Наши компоненты:
1. Инструменты поиска в интернете
2. Изначальный агент ответчик, создающий ответ из своих знаний (и пишущий критику на свои ответы)
3. Ревьювер: дополняет ответ (и критикует дополнения) исходя из новой полученной информации

Сначала объявим инструменты поиска

#### Объявление инструментов поиска


In [3]:
tool_executor = ToolExecutor([create_search_tool(llm)])

Инструменты вызываются в контексте графа. Создаем функцию, которая принимает контекст и ищет исходя из него информацию.

In [4]:
def execute_tools(state: List[BaseMessage]) -> List[BaseMessage]:
    messages = state["messages"]
    tool_invocation: AIMessage = messages[-1]
    tool_invocations = []
    for query in tool_invocation.additional_kwargs["function_call"]["arguments"][
        "search_queries"
    ][:]:
        tool_invocations.append(
            ToolInvocation(
                tool="search_engine",
                tool_input=query,
            )
        )

    outputs = tool_executor.batch(tool_invocations)
    arguments = tool_invocation.additional_kwargs["function_call"]["arguments"]
    reflection = arguments["reflection"]
    search_results = []
    for output, invocation in zip(outputs, tool_invocations):
        search_results.append(
            f"""Поисковой запрос: "{invocation.tool_input}"\n\n\n{output}\n"""  # noqa
        )
    search_results = "\n".join(search_results)
    result = f"""Вопрос: "{state["question"]}"
Ответ:
---
{arguments['answer']}
---
Критика: "{reflection}"

Добавь в ответ новую информацию, ответь на критику и напиши дополнительную важную информацию из результатов поиска ниже:
* Ответ должен отвечать на критику, добавь в него важную информацию из результатов поиска
* Покритикуй свой новый ответ и создай новые поисковые запросы

Результаты поиска:
---
{search_results}
---

Добавь в ответ новую информацию, ответь на критику и напиши дополнительную важную информацию из результатов поиска выше
Используй функцию ReviseAnswer.
"""  # noqa
    return {
        "messages": [HumanMessage(content=result)],
        "iteration": state["iteration"] + 1,
        "question": state["question"],
    }

### Изначальный ответчик

In [5]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """Ты бот эксперт-исследователь. Ты помогаешь пользователю в его исследованиях.
Текущее время: {time}

1. {first_instruction}
2. Постоянно размышляй и критикуй свои мысли.
3. Рекомендуй поисковые запросы для получения нужной информации.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)

In [6]:
class AnswerQuestion(BaseModel):
    """Функция ответа на вопрос пользователя.
    Возвращает результаты поиска по твоим поисковым запросам. Дополни ответ исходя из них
    """

    answer: str = Field(description="Развернутый ответ на вопрос пользователя")
    reflection: str = Field(
        description="Критика твоего текущего ответа. Что в нем можно дополнить?"
    )
    search_queries: List[str] = Field(
        description="1–3 поисковых запроса, которые позволят решить критику и дополнить твой текущий ответ"  # noqa
    )


initial_answer_chain = actor_prompt_template.partial(
    first_instruction="Дай развернутый ответ на вопрос пользователя с помощью функции AnswerQuestion."
) | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")
validator = PydanticOutputFunctionsParser(pydantic_schema=AnswerQuestion)

### Важно
Хочу заметить, что это очень полезный класс, позволяющий заполнять аргументы функции, если гигачат их пропустил.
А он пока что любит пропускать некоторые аргументы, даже если они указаны как required

In [ ]:
class ResponderWithRetries:
    def __init__(self, runnable, validator):
        self.runnable = runnable
        self.validator = validator

    @traceable
    def respond(self, state):
        response = []
        messages = state["messages"]
        for attempt in range(5):
            try:
                response = self.runnable.invoke({"messages": state["messages"]})
                self.validator.invoke(response)
                return {
                    "messages": messages + [response],
                    "iteration": state["iteration"] + 1,
                    "references": state["references"]
                    + response.additional_kwargs["function_call"]["arguments"].get(
                        "references", []
                    ),
                }
            except Exception as e:
                message = HumanMessage(content=repr(e))
                if response.response_metadata["finish_reason"] != "error":
                    state["messages"] += [message]
        return {
            "messages": messages + [response],
            "iteration": state["iteration"] + 1,
        }

In [7]:
first_responder = ResponderWithRetries(
    runnable=initial_answer_chain, validator=validator
)

Пример работы

In [18]:
example_question = "Как победить глобальное потепление"
state = {
    'messages': [HumanMessage(content=example_question)],
    'references': [],
    'iteration': 0,
    "question": example_question
}
state = {**state, **first_responder.respond(state)}
state['messages'][-1].additional_kwargs['function_call']['arguments']

Giga generation stopped with reason: function_call
Giga generation stopped with reason: function_call


{'answer': 'Для победы над глобальным потеплением необходимо принять комплекс мер, включающих снижение выбросов парниковых газов, увеличение зеленых насаждений, разработку и внедрение возобновляемых источников энергии, повышение энергоэффективности и многое другое.',
 'reflection': 'Хотя ответ общий, он не предлагает конкретных стратегий или планов действий. Нужно более подробно описать, какие именно шаги нужно предпринять для борьбы с глобальным потеплением.',
 'search_queries': ['меры по борьбе с глобальным потеплением',
  'стратегии снижения выбросов парниковых газов',
  'развитие возобновляемых источников энергии']}

### Ревьювер

In [19]:
revise_instructions = """Дополни существующий ответ, ответь на критику с помощью функции ReviseAnswer.
* Перепиши ответ, отвечая на свою критику, добавь информацию из результатов поиска ниже
* Покритикуй свой новый дополненный ответ и создай новые поисковые запросы
"""  # noqa


# Расширяем функцию ответа на вопрос, для того, чтобы мы могли добавлять источники к ответам
class ReviseAnswer(AnswerQuestion):
    """Функция ответа на вопрос пользователя. Дополни предыдущий ответ, ответь на критику.
    Сделай новую критику своего дополненного ответа и создай новые ссылки.
    Возвращает результаты поиска. Дополни дальше свой ответ исходя из них"""

    answer: str = Field(description="Новый ответ на вопрос пользователя")
    reflection: str = Field(
        description="Критика нового ответа. Что в нем можно дополнить?"
    )
    search_queries: List[str] = Field(
        description="1–3 поисковых запроса, которые позволят решить критику и дополнить твой новый ответ"  # noqa
    )
    references: List[str] = Field(
        description="Ссылки на источники информации, которые ты использовал в своем ответе."
    )


revision_chain = actor_prompt_template.partial(
    first_instruction=revise_instructions
) | llm.bind_tools(tools=[ReviseAnswer], tool_choice="ReviseAnswer")
revision_validator = PydanticOutputFunctionsParser(pydantic_schema=ReviseAnswer)

revisor = ResponderWithRetries(runnable=revision_chain, validator=revision_validator)

In [20]:
state = {**state, **execute_tools(state)}
state = {**state, **revisor.respond(state)}
state['messages'][-1].additional_kwargs['function_call']['arguments']

Giga generation stopped with reason: function_call
Giga generation stopped with reason: function_call


{'answer': 'Чтобы победить глобальное потепление, необходим комплексный подход, включающий сокращение выбросов парниковых газов, увеличение зеленых насаждений, развитие и внедрение возобновляемых источников энергии, повышение энергоэффективности и многое другое. В частности, правительства принимают политики для подготовки к экономическим потрясениям, связанным с изменением климата, включая требование доступа граждан к страхованию от пожаров и раскрытие информации банками о потенциальном влиянии изменения климата на их инвестиции. Также важны стратегии мониторинга и сокращения выбросов парниковых газов, таких как использование энергоэффективности, возобновляемых источников энергии, снижение выбросов метана и других подходов. Администрация Байдена-Харриса выпустила стратегию мониторинга парниковых газов, которая включает использование различных инструментов анализа и данных для учета выбросов парниковых газов. С развитием возобновляемых источников энергии, глобальная вместимость увеличил

## Построение графа
Соединяем между собой все компоненты

In [21]:
from langgraph.graph import END, StateGraph


class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    iteration: int
    question: str
    references: list[str]


MAX_ITERATIONS = 6
builder = StateGraph(AgentState)
builder.add_node("draft", first_responder.respond)
builder.add_node("execute_tools", execute_tools)
builder.add_node("revise", revisor.respond)
# draft -> execute_tools
builder.add_edge("draft", "execute_tools")
# execute_tools -> revise
builder.add_edge("execute_tools", "revise")

# Define looping logic:


def event_loop(state) -> str:
    # in our case, we'll just stop after N plans
    if state["iteration"] > MAX_ITERATIONS:
        return END
    return "execute_tools"


# revise -> execute_tools OR end
builder.add_conditional_edges("revise", event_loop)
builder.set_entry_point("draft")
graph = builder.compile()

In [ ]:
import sys

stdout = sys.stdout
# reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [25]:
question = "Как победить глобальное потепление?"
events = graph.stream(
    {
        "messages": [HumanMessage(content=question)],
        "iteration": 0,
        "question": question,
        "references": [],
    }
)
for i, step in enumerate(events):
    node, output = next(iter(step.items()))
    print(f"## {i+1}. {node}")
    if node in ["draft", "revise"]:
        args = output["messages"][-1].additional_kwargs["function_call"]["arguments"]
        print(f"Ответ: {args['answer']}")
        print(f"Критика: {args['reflection']}")
        print(f"Что будем искать: {args['search_queries']}")
    else:
        print("Произошел поиск")
    print("---")

Giga generation stopped with reason: function_call


## 1. draft
Ответ: Для победы над глобальным потеплением необходимо снизить выбросы парниковых газов, внедрять экологически чистые технологии, повысить энергоэффективность и использовать возобновляемые источники энергии. Также важно провести масштабную кампанию по просвещению населения о проблемах экологии и важности снижения углеродного следа. Но это только начало - чем больше людей будет вовлечено в процесс борьбы с глобальным потеплением, тем лучше.
Критика: Хотя ответ кажется полным, но он не указывает на конкретные действия или меры, которые могут быть предприняты отдельными людьми или организациями. Может быть, стоит добавить информацию о том, как каждый человек может внести свой вклад в борьбу с глобальным потеплением.
Что будем искать: ['как уменьшить свой углеродный след', 'экологические привычки', 'возобновляемые источники энергии']
---
## 2. execute_tools
Произошел поиск
---


Giga generation stopped with reason: function_call


## 3. revise
Ответ: Для победы над глобальным потеплением необходимо внедрить различные стратегии, включая снижение выбросов парниковых газов, повышение энергоэффективности, использование возобновляемых источников энергии и проведение кампании по экологическому просвещению. Кроме того, каждый человек может внести свой вклад в борьбу с глобальным потеплением, снизив свой углеродный след. Для этого можно сократить потребление воды, использовать меньше пластика, компостировать пищевые отходы, использовать общественный транспорт вместо автомобиля, повышать энергоэффективность дома и использовать возобновляемые источники энергии. Все эти меры помогают снизить нагрузку на окружающую среду и способствуют борьбе с глобальным потеплением.
Критика: Мой новый ответ включает информацию о конкретных действиях, которые могут быть предприняты для борьбы с глобальным потеплением, и подчеркивает важность каждого человека в этом процессе. Он также содержит ссылки на источники, подтверждающие представлен

Giga generation stopped with reason: error


content='' response_metadata={'token_usage': Usage(prompt_tokens=2049, completion_tokens=562, total_tokens=2611), 'model_name': 'GigaChat-Pro:2.2.25.3', 'finish_reason': 'error'}


Giga generation stopped with reason: function_call


## 5. revise
Ответ: Для победы над глобальным потеплением необходимо внедрить различные стратегии, включая снижение выбросов парниковых газов, повышение энергоэффективности, использование возобновляемых источников энергии и проведение кампании по экологическому просвещению. Важными мерами для каждого человека являются снижение потребления воды, использование меньше пластика, компостирование пищевых отходов, использование общественного транспорта вместо автомобиля, повышение энергоэффективности дома и использование возобновляемых источников энергии. Согласно Всемирному экономическому форуму, эффективными шагами в борьбе с глобальным потеплением являются переход от использования ископаемого топлива к возобновляемым источникам энергии, а также увеличение производства возобновляемой энергии и энергоэффективности к 2030 году. Кроме того, как сообщает Агентство по охране окружающей среды США, было принято решение о постепенном снижении производства и потребления ГФУ в США на 85% в течение сл

Giga generation stopped with reason: function_call
Giga generation stopped with reason: function_call


## 7. revise
Ответ: Для победы над глобальным потеплением необходимо внедрить различные стратегии, включая снижение выбросов парниковых газов, повышение энергоэффективности, использование возобновляемых источников энергии и проведение кампании по экологическому просвещению. Важными мерами для каждого человека являются снижение потребления воды, использование меньше пластика, компостирование пищевых отходов, использование общественного транспорта вместо автомобиля, повышение энергоэффективности дома и использование возобновляемых источников энергии. Согласно Всемирному экономическому форуму, эффективными шагами в борьбе с глобальным потеплением являются переход от использования ископаемого топлива к возобновляемым источникам энергии, а также увеличение производства возобновляемой энергии и энергоэффективности к 2030 году. Также как сообщает Агентство по охране окружающей среды США, было принято решение о постепенном снижении производства и потребления ГФУ в США на 85% в течение следующи

In [26]:
args = step[END]["messages"][-1].additional_kwargs["function_call"]["arguments"]
print(f"{args['answer']}")
print("\n".join(step[END]["references"]))

Для победы над глобальным потеплением необходимо внедрить различные стратегии, включая снижение выбросов парниковых газов, повышение энергоэффективности, использование возобновляемых источников энергии и проведение кампании по экологическому просвещению. Важными мерами для каждого человека являются снижение потребления воды, использование меньше пластика, компостирование пищевых отходов, использование общественного транспорта вместо автомобиля, повышение энергоэффективности дома и использование возобновляемых источников энергии. Согласно Всемирному экономическому форуму, эффективными шагами в борьбе с глобальным потеплением являются переход от использования ископаемого топлива к возобновляемым источникам энергии, а также увеличение производства возобновляемой энергии и энергоэффективности к 2030 году. Также как сообщает Агентство по охране окружающей среды США, было принято решение о постепенном снижении производства и потребления ГФУ в США на 85% в течение следующих 15 лет. Среди техн

## Выводы
1. Агент жертвует временем выполнения в пользу качества и полноты ответа.
2. Критика позволяет направлять агента, для поиска нужной информации и формирования ответа